In [1]:
from graphnics import *
from xii import *

import sys
sys.path.append('../data/')
from generate_arterial_tree import *

import time

Missing HsMG for fract norm computing


# Time profiling of graphnics

The Network Stokes model is posed with a global pressure $p \in P^1(\Lambda)$ and edge-wise cross-section fluxes $\hat{q}_i \in P^2(\Lambda_i)$. For this reason the Network Stokes model gets computationally expensive for networks with a large number of edges.

In the below code we examine this in more detail:

In [2]:
# Generations of arterial tree
Ns = [1, 2, 3, 4, 5, 6, 7, 8]

# Fix tree branch direction split
signs = np.tile([-1,1], 200).tolist()

print('| N | n_e| assemble  | solve| total |')
print('| -- | -- |-- |-- | -- |')

for N in Ns:
    G = make_arterial_tree(N, directions = signs, gam=0.9, lmbda = 8, radius0=0.1)
    G.make_mesh(0)
    
    for e in G.edges():
        G.edges()[e]['Ainv'] = Constant(1)
        G.edges()[e]['Res'] = Constant(1)
    
    # Profile assembly and solver times
    t_start = time.time()

    # Setup and assemble network
    model = NetworkStokes(G, p_bc = Expression('x[1]', degree=2))

    W = model.W
    a = model.a_form()
    L = model.L_form()

    A, b = map(ii_assemble, (a,L))
    A, b = map(ii_convert, (A,b))
    
    t_assemble = time.time() - t_start
    
    # Solve network equations
    qp = ii_Function(W)
    solver = LUSolver(A, 'mumps')
    solver.solve(qp.vector(), b)

    t_solve = time.time()-t_start-t_assemble
        
    t_total = t_solve + t_assemble 
    
    print(f'| {N} | {G.num_edges} | {t_assemble:1.2f}s | {t_solve:1.2f}s | {t_total:1.2f}s')

| N | n_e| assemble  | solve| total |
| -- | -- |-- |-- | -- |
| 1 | 1 | 0.04s | 0.00s | 0.04s
| 2 | 3 | 0.05s | 0.00s | 0.05s
| 3 | 7 | 0.13s | 0.00s | 0.13s
| 4 | 15 | 0.37s | 0.00s | 0.38s
| 5 | 31 | 1.32s | 0.01s | 1.33s
| 6 | 63 | 3.94s | 0.02s | 3.95s
| 7 | 127 | 14.30s | 0.04s | 14.34s
| 8 | 255 | 56.79s | 0.09s | 56.89s


## Runtimes

Executing the above code yields the following table:

| N | assemble  | solve| total |
| -- | -- |-- |-- |
| 1 | 0.04s | 0.00s | 0.04s
| 2 | 0.07s | 0.00s | 0.07s
| 3 | 0.18s | 0.00s | 0.18s
| 4 | 0.44s | 0.00s | 0.44s
| 5 | 1.17s | 0.01s | 1.17s
| 6 | 4.06s | 0.01s | 4.07s
| 7 | 14.65s | 0.04s | 14.69s
| 8 | 55.77s | 0.09s | 55.86s